In [1]:
import tensorflow as tf
import coremltools as ct

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

print(tf.__version__)

2.3.1


In [2]:
def normalize(x):
    x = 2 * ( (x - np.min(x)) / (np.max(x) - np.min(x)) ) - 1
    return x


def demean(x):
    x = x - np.mean(x)
    return x

In [3]:
clss = [0,1,2,4] # mask_off, normal_breathing, talking, cough
X_train, Y_train, X_test, Y_test = [], [], [], []


data_files = []
for root, subdirs, files in os.walk('data/train'):
    if len(files) == 0: continue
    
    for file in files:
        data_files.append(f'{root.split("/")[-1]}/{file}')

print(data_files)


for file in data_files:
    train = np.load(f'data/train/{file}')
    test = np.load(f'data/test/{file}')

    # normalize sample
    for s in train:
        sample = np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        X_train.append(sample)
        if 'mask_off' in file: Y_train.append(0)
        elif 'normal_breathing' in file: Y_train.append(1)
        elif 'talking' in file: Y_train.append(2)
        elif 'cough' in file: Y_train.append(3)

    for s in test:
        X_test.append(
            np.vstack((
                demean(s[0]),
                demean(s[1])
            ))
        )
        if 'mask_off' in file: Y_test.append(0)
        elif 'normal_breathing' in file: Y_test.append(1)
        elif 'talking' in file: Y_test.append(2)
        elif 'cough' in file: Y_test.append(3)
    
X_train, Y_train, X_test, Y_test = np.array(X_train, dtype='float64'), np.array(Y_train, dtype='float64'), np.array(X_test, dtype='float64'), np.array(Y_test, dtype='float64')

['talking/s_2021-02-10 18:34:39.763.npy', 'talking/s_2021-02-10 16:36:38.417.npy', 'cough/s_2021-02-10 22:18:44.719.npy', 'cough/s_2021-02-10 20:53:33.741.npy', 'cough/s_2021-02-10 22:18:17.501.npy', 'cough/s_2021-02-10 20:57:59.617.npy', 'normal_breathing/s_2021-02-10 15:11:17.966.npy', 'normal_breathing/s_2021-02-10 18:12:14.117.npy', 'mask_off/s_2021-02-10 16:24:58.357.npy', 'mask_off/s_2021-02-10 18:27:53.484.npy']


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2,51)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4)
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True ),
    metrics=['accuracy']
)

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 102)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1648      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 1,716
Trainable params: 1,716
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(np.nan_to_num(X_train), Y_train, epochs=100, batch_size=len(X_train))

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 1.4798 - accuracy: 0.2459
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 1.4728 - accuracy: 0.2511
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 1.4660 - accuracy: 0.2622
Epoch 4/100
1/1 [==============================] - 0s 656us/step - loss: 1.4594 - accuracy: 0.2754
Epoch 5/100
1/1 [==============================] - 0s 765us/step - loss: 1.4530 - accuracy: 0.2828
Epoch 6/100
1/1 [==============================] - 0s 823us/step - loss: 1.4466 - accuracy: 0.2865
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 1.4404 - accuracy: 0.2909
Epoch 8/100
1/1 [==============================] - 0s 756us/step - loss: 1.4344 - accuracy: 0.2975
Epoch 9/100
1/1 [==============================] - 0s 693us/step - loss: 1.4285 - accuracy: 0.2997
Epoch 10/100
1/1 [==============================] - 0s 723us/step - loss: 1.4227 - accuracy: 0.3019
Epoch 11/100
1/1 

In [8]:
test_loss, test_acc = model.evaluate(np.nan_to_num(X_test),  Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

18/18 - 0s - loss: 0.8177 - accuracy: 0.9219

Test accuracy: 0.921875


In [9]:
model.save('simple_nn')
mlmodel = ct.convert('simple_nn') 

Translating MIL ==> MLModel Ops: 100%|██████████| 9/9 [00:00<00:00, 5101.87 ops/s]


In [10]:
mlmodel.save('simple_nn.mlmodel')